In [1]:
import numpy as np
data_file = "/home/sumit/Desktop/project17/resume_parser/SkrResumeParser/Data/resume_segments/Level0/all.txt"

In [2]:
from SkrResumeParser.VectorProcessing.GramGloveSentenceVector import GramGloveSentenceVector
gsv = GramGloveSentenceVector(data_file, dimension=50)
sen2vec = gsv.get_5gram_sentenceVector()
np.random.shuffle(sen2vec)
print len(sen2vec), len(sen2vec[0])

35467 2


In [3]:
def train_test_split(data):
    train_data = np.array(data[0:int(len(data) * 0.8)])
    test_data = np.array(data[int(len(data) * 0.8):])
    train_x = np.array(list(train_data[:, 0]))
    train_y = np.array(list(train_data[:, 1]))
    test_x = np.array(list(test_data[:, 0]))
    test_y = np.array(list(test_data[:, 1]))

    return train_x, test_x, train_y, test_y

In [4]:
train_x, test_x, train_y, test_y = train_test_split(sen2vec)
#train_x = np.reshape(train_x, (len(train_x), len(train_x[0]), 1))
#test_x = np.reshape(test_x, (len(test_x), len(test_x[0]), 1))
print len(train_x), len(test_x), len(train_x[0])

28373 7094 250


In [5]:
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 1024, kernel_initializer = 'uniform', activation = 'relu', input_dim = 250))

# Adding the second hidden layer
classifier.add(Dense(units = 1024, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(train_x, train_y, batch_size = 100, epochs = 30)

Using TensorFlow backend.


Epoch 1/30
28373/28373 [==============================] - 16s - loss: 0.6335 - acc: 0.8106    
Epoch 2/30
28373/28373 [==============================] - 17s - loss: 0.3936 - acc: 0.8857    
Epoch 3/30
28373/28373 [==============================] - 16s - loss: 0.3100 - acc: 0.9086    
Epoch 4/30
28373/28373 [==============================] - 18s - loss: 0.2407 - acc: 0.9267    
Epoch 5/30
28373/28373 [==============================] - 18s - loss: 0.1954 - acc: 0.9384    
Epoch 6/30
28373/28373 [==============================] - 17s - loss: 0.1597 - acc: 0.9481    
Epoch 7/30
28373/28373 [==============================] - 17s - loss: 0.1236 - acc: 0.9590    
Epoch 8/30
28373/28373 [==============================] - 16s - loss: 0.1033 - acc: 0.9663    
Epoch 9/30
28373/28373 [==============================] - 16s - loss: 0.0789 - acc: 0.9735    
Epoch 10/30
28373/28373 [==============================] - 17s - loss: 0.0697 - acc: 0.9771    
Epoch 11/30
28373/28373 [========================

In [6]:
y_pred = classifier.predict(test_x)

correct = np.equal(np.argmax(y_pred, 1), np.argmax(test_y, 1))
accuracy = np.mean(correct)
print("Accuracy", accuracy)
print "pred: ", np.argmax(y_pred, 1)[0:30]
print "real: ", np.argmax(test_y, 1)[0:30]
print "pred: ", np.argmax(y_pred, 1)[30:65]
print "real: ", np.argmax(test_y, 1)[30:65]
print "pred: ", np.argmax(y_pred, 1)[65:100]
print "real: ", np.argmax(test_y, 1)[5000:]
print "real: ", np.argmax(y_pred, 1)[5000:]

('Accuracy', 0.90950098674936564)
pred:  [1 5 1 1 1 1 0 5 5 6 1 0 1 0 5 1 5 6 1 1 7 5 1 1 0 1 1 1 1 1]
real:  [1 5 1 1 1 1 0 5 5 0 1 0 1 0 5 1 5 4 1 1 7 5 1 1 0 1 1 1 1 1]
pred:  [7 2 1 5 5 0 5 1 7 2 1 0 5 5 5 5 1 7 1 1 0 1 5 0 4 1 0 2 5 1 5 1 2 1 5]
real:  [5 2 1 5 5 0 5 1 2 2 1 0 5 5 5 5 1 7 1 1 0 1 5 6 0 1 0 2 5 1 7 1 2 1 5]
pred:  [5 5 2 1 5 0 2 1 0 0 1 0 0 1 0 1 0 1 7 1 7 5 1 1 2 1 1 5 1 1 1 0 1 1 1]
real:  [2 1 5 ..., 0 7 0]
real:  [2 1 0 ..., 0 7 0]


In [7]:
resume_file = "/home/sumit/Desktop/project17/resume_parser/SkrResumeParser/Data/resume_segments/Test/sample.txt"
test_gsv = GramGloveSentenceVector(resume_file, dimension=50, training=False)
test_sen2vec = test_gsv.get_5gram_sentenceVector()
print len(test_sen2vec), len(test_sen2vec[0])

126 250


In [8]:
resume_predict = classifier.predict(test_sen2vec)
print np.argmax(resume_predict, 1)
dirlist = ['basic', 'experience', 'education', 'certificate', 'extra', 'skills', 'projects','summary', 'mimc']
with open(resume_file, 'r') as f:
    content = f.readlines()
count = 0
for line in content:
    print line ,'=>', dirlist[np.argmax(resume_predict, 1)[count]]
    count += 1

[0 5 5 5 7 1 4 4 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 3 1 5 5 7 7 7 5 5 5 5 5 5 5
 5 1 4 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 1 6 2 1 0 0 0 5 5 7 1 4 4 1 1 1
 2 2 2 2 2 2 2 2 2 2 2 2 3 1 5 5 7 7 7 5 5 5 5 5 5 5 5 1 4 1 1 5 1 1 1 1 1
 1 1 1 1 1 1 1 3 3 3 1 6 2 0 0]
SUMIT RATHORE
=> basic
Web Developer
=> skills
8800904192       sumitrathore1313@gmail.com
=> skills
CAREER OBJECTIVE
=> skills
Self-motivated and hardworking fresher seeking for an opportunity to work in a challenging environment to prove my skills and utilize my knowledge & intelligence in the growth of the organization.
=> summary
WORK EXPERIENCE
=> experience
Arbunize, Delhi, Delhi
=> extra
Internship, Jun 2017 – Jul 2017
=> extra
Web Developer
=> experience
Design, build, or maintain web sites, using authoring or scripting languages, content creation tools, management tools, and digital media.
=> experience

=> experience
EDUCATION
=> education
Ims Engineering College, Ghaziabad, Uttar Pradesh
=> education
B Tech Computer Scienc

In [9]:
classifier.save('/home/sumit/Desktop/project17/resume_parser/SkrResumeParser/Data/resume_segments/Test/5GramClassifierOther')